The first two targets the signed constructor. This works.

- Guid::New [Int32, Int16, Int16, Byte, Byte, Byte, Byte, Byte, Byte, Byte, Byte](https://docs.microsoft.com/en-us/dotnet/api/system.guid.-ctor?view=net-6.0#system-guid-ctor(system-int32-system-int16-system-int16-system-byte-system-byte-system-byte-system-byte-system-byte-system-byte-system-byte-system-byte))

The one that breaks

- Guid::New [UInt32, UInt16, UInt16, Byte, Byte, Byte, Byte, Byte, Byte, Byte, Byte](https://docs.microsoft.com/en-us/dotnet/api/system.guid.-ctor?view=net-6.0#system-guid-ctor(system-uint32-system-uint16-system-uint16-system-byte-system-byte-system-byte-system-byte-system-byte-system-byte-system-byte-system-byte))


I'm not sure, but it kind of looks like it's converting a `Uint32` to an `int32` *before* the constructor override is chosen?
```cs
Convert.ThrowInt32OverflowException()
UInt32.System.IConvertible.ToInt32(IFormatProvider provider)
Convert.ChangeType(Object value, Type conversionType, IFormatProvider provider)
```

In [ ]:
# Import-Module 'classExplorer', 'ninmonkey.console' *>$null -pass
# Import-Module 'classExplorer', 'ninmonkey.console' *>$null -pass
[byte[]]$bytes = [Convert]::FromHexString('ff' * 8 -join '') # euivalent: ('ffffffff' + 'ffffffff') #

h1 'signed, all low'
[int32]$a = 500
[int16]$b = 500
[int16]$c = 500
[Guid]::new($a, $b, $c, $bytes)

h1 'unsigned, all low'
[uint32]$ua = 500
[uint16]$ub = 500
[uint16]$uc = 500
[Guid]::new($ua, $ub, $uc, $bytes)


h1 'unsigned, max'  # Throws
[uint32]$ua = [uint32]::maxValue
[uint16]$ub = [uint16]::maxValue
[uint16]$uc = [uint16]::maxValue
[Guid]::new( $ua, $ub, $uc, $bytes)
h1 'extra explicit'
[Guid]::new( [uint32]$ua, [uint16]$ub, [uint16]$uc, $bytes)


# signed, all low


Guid
----
000001f4-01f4-01f4-ffff-ffffffffffff

# unsigned, all low

000001f4-01f4-01f4-ffff-ffffffffffff

# unsigned, max

MethodException: 
Line |
  22 |  [Guid]::new( $ua, $ub, $uc, $bytes)
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Cannot convert argument "a", with value: "4294967295", for ".ctor" to type "System.Int32": "Cannot convert value "4294967295" to type "System.Int32". Error: "Value was either too large or too small for an Int32.""

# extra explicit

MethodException: 
Line |
  24 |  [Guid]::new( [uint32]$ua, [uint16]$ub, [uint16]$uc, $bytes)
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Cannot convert argument "a", with value: "4294967295", for ".ctor" to type "System.Int32": "Cannot convert value "4294967295" to type "System.Int32". Error: "Value was either too large or too small for an Int32.""



## Showing errors with ansi colors on github

It seems like 

In [ ]:
#[Guid]::new( [uint32]$ua, [uint16]$ub, [uint16]$uc, $bytes)
$error[0] | get-error


Exception             : 
    Type           : System.Management.Automation.MethodException
    ErrorRecord    : 
        Exception             : 
            Type    : System.Management.Automation.ParentContainsErrorRecordException
            Message : Cannot convert argument "a", with value: "4294967295", for 
".ctor" to type "System.Int32": "Cannot convert value "4294967295" to type "System.Int32". Error: 
"Value was either too large or too small for an Int32.""
            HResult : -2146233087
        CategoryInfo          : NotSpecified: (:) [], ParentContainsErrorRecordException
        FullyQualifiedErrorId : MethodArgumentConversionInvalidCastArgument
        InvocationInfo        : 
            ScriptLineNumber : 1
            OffsetInLine     : 1
            HistoryId        : -1
            Line             : [Guid]::new( [uint32]$ua, [uint16]$ub, [uint16]$uc, $bytes)
            PositionMessage  : At line:1 char:1
                               + [Guid]::new( [uint32]$ua,